In [1]:
import os 
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
     

In [2]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir =r"C:\food\pythonProject\image data\corn\train"
test_dir=r"C:\food\pythonProject\image data\corn\val"

In [3]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*")) 
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

4 Classes
3080 Train images
772 Test images


In [4]:

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [5]:
input_shape=(256,256,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(256,256),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(256,256),batch_size=32)
     

Found 3080 images belonging to 4 classes.
Found 772 images belonging to 4 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
                       test_dir, # same directory as training data
                       target_size=(256, 256),
                       batch_size=32)

Found 772 images belonging to 4 classes.


In [7]:
# Define hyperparameters
learning_rate = 5e-5
train_batch_size = 256
eval_batch_size = 256
num_epochs = 30

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers in the pretrained model
freeze_until_layer = 'block4_conv3'
for layer in pretrained_model.layers:
    if layer.name == freeze_until_layer:
        break
    layer.trainable = False #weights of this layer will not be updated during training

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: learning_rate * (1 - epoch / num_epochs),
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_scheduler]
)



Epoch 1: LearningRateScheduler setting learning rate to 5e-05.
Epoch 1/30
12/12 [==============================] - 163s 13s/step - loss: 0.8162 - accuracy: 0.6694 - val_loss: 0.4924 - val_accuracy: 0.8333 - lr: 5.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 4.8333333333333334e-05.
Epoch 2/30
12/12 [==============================] - 155s 13s/step - loss: 0.3386 - accuracy: 0.8750 - val_loss: 0.2194 - val_accuracy: 0.8750 - lr: 4.8333e-05

Epoch 3: LearningRateScheduler setting learning rate to 4.666666666666667e-05.
Epoch 3/30
12/12 [==============================] - 157s 13s/step - loss: 0.2184 - accuracy: 0.8984 - val_loss: 0.1997 - val_accuracy: 0.8958 - lr: 4.6667e-05

Epoch 4: LearningRateScheduler setting learning rate to 4.5e-05.
Epoch 4/30
12/12 [==============================] - 128s 10s/step - loss: 0.1964 - accuracy: 0.9193 - val_loss: 0.1941 - val_accuracy: 0.9062 - lr: 4.5000e-05

Epoch 5: LearningRateScheduler setting learning rate to 4.33333333333333

In [8]:
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


25/25 [==============================] - 134s 5s/step - loss: 0.0691 - accuracy: 0.9754
Test Loss: 0.06911176443099976
Test Accuracy: 0.9753885865211487


In [12]:
model.save("corn.h5")